In [1]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# must enter API key
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-langsmith-groq"

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="openai/gpt-oss-120b",
            temperature = 0.2)

In [3]:
prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""

In [4]:
from langchain_core.prompts import ChatPromptTemplate

# passing the template to the LangChain model
prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{query}"),
])

In [5]:
prompt_template.input_variables

['context', 'query']

In [6]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

In [ ]:
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt), # -> prompt and context
    HumanMessagePromptTemplate.from_template("{query}") # -> user input
])

In [8]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

In [10]:
pipeline = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"]
    }
    | prompt_template
    | llm
)

In [11]:
context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

In [12]:
pipeline.invoke({"query": query, "context": context})

AIMessage(content='Aurelio\u202fAI is an AI‑focused company that builds tools and services for AI engineers, especially those working with language models. Their core activities include:\n\n* **Developing open‑source frameworks** – most notably **Semantic\u202fRouter** and **Semantic\u202fChunkers**, which help with routing and chunking of textual data.\n* **Providing an AI Platform** – a suite of tooling that streamlines the end‑to‑end development, deployment, and management of language‑model‑based applications.\n* **Offering development services** – helping other organizations design, build, and bring AI‑driven products to market, leveraging their deep expertise in AI agents and information‑retrieval techniques.\n* **LangChain expertise** – since September\u202f2024, Aurelio\u202fAI has been recognized as a LangChain Expert, reflecting a strong track record of delivering solutions built on the LangChain ecosystem.', additional_kwargs={'reasoning_content': 'We need to answer based on 

In [13]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

examples = [
    {"input": "Here is query #1", "output": "Here is the answer #1"},
    {"input": "Here is query #2", "output": "Here is the answer #2"},
    {"input": "Here is query #3", "output": "Here is the answer #3"},
]

In [14]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
# here is the formatted prompt
print(few_shot_prompt.format())

Human: Here is query #1
AI: Here is the answer #1
Human: Here is query #2
AI: Here is the answer #2
Human: Here is query #3
AI: Here is the answer #3


In [15]:
new_system_prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Always answer in markdown format. When doing so please
provide headers, short summaries, follow with bullet
points, then conclude.

Context: {context}
"""

prompt_template.messages[0].prompt.template = new_system_prompt

out = pipeline.invoke({"query": query, "context": context}).content
print(out)

## Overview  
Aurelio AI is an AI‑focused company that builds tools and services for **AI engineers**, especially those working with **language models** and **AI agents**.

### What Aurelio AI Offers
- **Open‑source frameworks**
  - **Semantic Router** – helps route queries and responses based on semantic similarity.  
  - **Semantic Chunkers** – splits text into meaningful chunks for retrieval‑augmented generation.
- **AI Platform**
  - Provides a suite of tooling that streamlines the development, testing, and deployment of language‑model‑centric applications.
- **Professional Services**
  - Offers development and consulting services to help organizations bring their AI products to market quickly and reliably.
- **LangChain Expertise**
  - Recognized as LangChain Experts (since September 2024) with a proven track record of delivering solutions built on the LangChain ecosystem.

### Core Focus Areas
- **Language AI** – building and optimizing large‑language‑model (LLM) applications.  


In [16]:
from IPython.display import display, Markdown

display(Markdown(out))

## Overview  
Aurelio AI is an AI‑focused company that builds tools and services for **AI engineers**, especially those working with **language models** and **AI agents**.

### What Aurelio AI Offers
- **Open‑source frameworks**
  - **Semantic Router** – helps route queries and responses based on semantic similarity.  
  - **Semantic Chunkers** – splits text into meaningful chunks for retrieval‑augmented generation.
- **AI Platform**
  - Provides a suite of tooling that streamlines the development, testing, and deployment of language‑model‑centric applications.
- **Professional Services**
  - Offers development and consulting services to help organizations bring their AI products to market quickly and reliably.
- **LangChain Expertise**
  - Recognized as LangChain Experts (since September 2024) with a proven track record of delivering solutions built on the LangChain ecosystem.

### Core Focus Areas
- **Language AI** – building and optimizing large‑language‑model (LLM) applications.  
- **AI Agents** – designing autonomous agents that can reason, retrieve information, and act.  
- **Information Retrieval** – leveraging semantic search and chunking to improve data access for LLMs.

## Conclusion  
Aurelio AI equips AI engineers with open‑source tools, a robust platform, and expert services to accelerate the creation of language‑model‑driven applications, while also providing deep expertise in LangChain and information retrieval.

In [17]:
no_cot_system_prompt = """
Be a helpful assistant and answer the user's question.

You MUST answer the question directly without any other
text or explanation.
"""

no_cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", no_cot_system_prompt),
    ("user", "{query}"),
])

In [19]:
query = (
    "How many keystrokes are needed to type the numbers from 1 to 5000?"
)

no_cot_pipeline = no_cot_prompt_template | llm
no_cot_result = no_cot_pipeline.invoke({"query": query}).content
print(no_cot_result)

18893


In [20]:
# Define the chain-of-thought prompt template
cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question, you must:

- List systematically and in precise detail all
  subproblems that need to be solved to answer the
  question.
- Solve each sub problem INDIVIDUALLY and in sequence.
- Finally, use everything you have worked through to
  provide the final answer.
"""

cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", cot_system_prompt),
    ("user", "{query}"),
])

cot_pipeline = cot_prompt_template | llm

In [21]:
cot_result = cot_pipeline.invoke({"query": query}).content
display(Markdown(cot_result))

**Step‑by‑step breakdown**

| Step | What we need to do | Numbers involved | How many numbers | Digits per number | Keystrokes contributed |
|------|-------------------|------------------|------------------|-------------------|------------------------|
| 1 | Identify the digit‑length groups that appear between 1 and 5000. | – | – | – | The groups are 1‑digit (1‑9), 2‑digit (10‑99), 3‑digit (100‑999), 4‑digit (1000‑5000). |
| 2 | Count how many 1‑digit numbers are in the range. | 1 – 9 | \(9\) | 1 | \(9 \times 1 = 9\) |
| 3 | Count how many 2‑digit numbers are in the range. | 10 – 99 | \(99-10+1 = 90\) | 2 | \(90 \times 2 = 180\) |
| 4 | Count how many 3‑digit numbers are in the range. | 100 – 999 | \(999-100+1 = 900\) | 3 | \(900 \times 3 = 2\,700\) |
| 5 | Count how many 4‑digit numbers are in the range **up to 4999**. | 1000 – 4999 | \(4999-1000+1 = 4\,000\) | 4 | \(4\,000 \times 4 = 16\,000\) |
| 6 | Add the final number, 5000, which is also 4‑digit. | 5000 | \(1\) | 4 | \(1 \times 4 = 4\) |
| 7 | Sum all contributions. | – | – | – | \(9 + 180 + 2\,700 + 16\,000 + 4 = 18\,893\) |

**Final answer**

The total number of keystrokes required to type every integer from **1** through **5000** (counting each digit as one keystroke) is  

\[
\boxed{18\,893\ \text{keystrokes}}.
\]